Докажите, что $k$-мерный гиперкуб $k$-связен с помощью теоремы Менгера.

$k$-мерным гиперкубом называется граф, в котором вершины — битовые строки длины $k$, а рёбра проведены между теми парами вершин, которые отличаются ровно в одном бите. На вход подаётся две различные битовых строки $A$ и $B$ длины $k<100$ через пробел. Выведите $k$ простых путей в $k$-мерном гиперкубе из $A$ в $B$, не пересекающихся по внутренним вершинам. Формат вывода: один путь на строку; путь – последовательности битовых строк, разделённая пробелами.

In [1]:
# моя попытка написать оптимизированный перебор путей на Python
def kpaths(input_str):
    # input_str - входная строка
    start, end = input_str.split()
    
    # оцениваем длину строк
    k = max(len(start), len(end))
    
    # для большей скорости работать будем не со строками, а с битовыми масками
    # поэтому преобразуем двоичные строки вида "0010..." в целые числа
    start, end = int(start,2), int(end,2)
    
    # начальная и конечная точка пути должны различаться,
    # иначе алгоритм отработает некорретно
    assert start != end, '%s and %s must be different' % (start, end)

    # powers - это массив битовых масок вида "10000", "00100", "00010", итд
    # он пригодится, поскольку при переборе путей мы будем последовательно
    # изменять по одному разряду текущего шага
    powers = [2**i for i in range(k)]
    
    # в множестве visited будем запоминать уже использованные в путях вершины
    visited = set()
    
    # в массиве result мы будем накапливать полученные пути
    # в принципе, можно было бы их просто печатать по мере получения,
    # но во время тестирования перебором различных входных строк
    # мне нужно было как-то запретить печать
    result = []

    # будем перебирать начальные вершины пути
    # сначала возьмём первую вершину без изменений,
    # потом изменим в ней крайний бит, потом следующий и т.д.
    # получится перебор k+1 начальных вершин, но одна из них будет
    # занята каким-то из путей в ходе поиска, и в итоге получится k путей
    for bit in [0] + powers:
        # изменяем один из битов начальной вершины и генерируем вторую вершину пути
        # (первой всегда будет start)
        val = start ^ bit

        # одна из потенциальных начальных вершин будет занята каким-то из путей в ходе поиска
        if val in visited:
            continue
        visited.add(val)

        # добавляем начальную вершину плюс вершину с изменённым битом (если изменяли) в путь
        path = [start] if val == start else [start, val]

        # крутимся, пока текущий путь не достиг конечной вершины
        while val != end:
            # целое число diff будет содержать единицы в тех битах,
            # которые отличаются между текущей и конечной вершинами
            diff = val ^ end

            # начинаем перебирать, куда бы двинуться из текущей вершины
            # мы пытаемся найти более короткий путь, то есть такой, в котором
            # число отличающихся от конечной вершины битов меньше на единицу
            for step in (val ^ (mask1 & diff) for mask1 in powers if mask1 & diff):
                # следующий шаг должен либо привести в конечную (step == end),
                # либо в еще не занятую другими путями (step not in visited) вершину
                if step == end or step not in visited:
                    val = step        # запоминаем новую вершину в текущей
                    visited.add(val)  # добавляем в набор уже занятых вершин
                    path.append(val)  # и добавляем её в путь
                    break             # выходим из цикла поиска шага
            else:
                # если все вершины уже заняты, значит в алгоритме какая-то ошибка
                raise RuntimeError('no more paths for %s' % input_str)

        # преобразуем путь из массива целых чисел в строковую форму и добавляем в результат
        result.append(' '.join(bin(val)[2:].rjust(k, '0') for val in path))

    # по теореме Менгера мы должны получить ровно k путей
    # проверим корректность результата
    assert len(result) == k, '%d paths for %s' % (len(result), input_str)
    return result

# распечатаем результат, по одному пути в каждой строке
#print(*kpaths(input()), sep='\n')

In [2]:
print(*kpaths('0 1'), sep='\n', end='\n\n')
print(*kpaths('000 011'), sep='\n', end='\n\n')
print(*kpaths('1010 0010'), sep='\n', end='\n\n')

0 1

000 001 011
000 010 011
000 100 101 111 011

1010 0010
1010 1011 0011 0010
1010 1000 0000 0010
1010 1110 0110 0010



In [3]:
%%timeit -r1 -n1

def long_test(k_max):
    from itertools import product
    def make_in(i, j, k):
        return ' '.join(bin(x)[2:].rjust(k, '0') for x in [i,j])
    for k in range(1, k_max+1):
        for i,j in product(range(2**k), range(2**k)):
            if i != j:
                kpaths(make_in(i, j, k))
    print('ok')

long_test(8)

ok
33.8 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
